In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Causal_Inference_Model/

/content/drive/MyDrive/Causal_Inference_Model


In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import Libraries

In [ ]:
import dvc.api
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import sys, os

In [45]:
sys.path.append(os.path.abspath('scripts/'))

In [56]:
from filehundle import LoadData
from plots import Plot
from overview import Overview
from preprocessing import PreProcess

Initialize Loaded Data

In [58]:
loader = LoadData()
plot = Plot()

INFO:logger:Successfully Instantiated load_data Class Object
INFO:logger:Successfully Instantiated Preprocessing Class Object


In [59]:
# Get URL from DVC
data_path = 'data/nb.csv'
repo = 'https://github.com/yonamg/Causal_Inference_Model/'
version = '31a6d9ca5315cecd1bf0cfee00a7e0'

In [ ]:
data_url = dvc.api.get_url(
    path=data_path,
    repo=repo,
    rev=version
)

In [62]:
trip_df = loader.read_csv("data/nb.csv")


In [63]:
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


In [64]:
miss_info = trip_df.isna().sum()
miss_info

Trip ID                0
Trip Origin            0
Trip Destination       0
Trip Start Time     1651
Trip End Time          1
dtype: int64

In [65]:
trip_df.rename(columns = {'Trip ID':'order_id',
                          'Trip Origin':"trip_origin",
                          'Trip Destination':'trip_destination',
                          'Trip Start Time':'trip_Start_time',
                          'Trip End Time':'trip_end_time'}, inplace = True)

In [66]:
trip_df.head()

,order_id,trip_origin,trip_destination,trip_Start_time,trip_end_time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


There are 1651 Trip which has no record of trip start time but has trip end time. Why the is a record for trip end time but not for trip start time? Do the driver register their end time only to do some tricks or is there system error for those sptecific trip without trip start time?

In [67]:
order_df = loader.read_csv("data/driver_locations_during_request.csv")
order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


In [68]:
order_df['order_origin'] = [str(x)+","+str(y) for x,y in zip(order_df.lat, order_df.lng)] 

In [69]:
order_df.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at,order_origin
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN,"6.6022066,3.2704649"
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN,"6.5920972,3.2874447"
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN,"6.5961334,3.2817841"
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN,"6.5961416,3.2805263"
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN,"6.6092317,3.2887999"


### Remove Outliers

In [70]:
order_df = order_df[order_df['lat']<7]
order_df = order_df[order_df['lng']<8]


### Merge the two tables

Use order_id and Trip ID columns to merge as the two columns are similar

In [71]:
df = order_df.merge(trip_df, how='inner', on='order_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1557739 entries, 0 to 1557738
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1557739 non-null  int64  
 1   order_id          1557739 non-null  int64  
 2   driver_id         1557739 non-null  int64  
 3   driver_action     1557739 non-null  object 
 4   lat               1557739 non-null  float64
 5   lng               1557739 non-null  float64
 6   created_at        0 non-null        float64
 7   updated_at        0 non-null        float64
 8   order_origin      1557739 non-null  object 
 9   trip_origin       1557739 non-null  object 
 10  trip_destination  1557739 non-null  object 
 11  trip_Start_time   1555717 non-null  object 
 12  trip_end_time     1557739 non-null  object 
dtypes: float64(4), int64(3), object(6)
memory usage: 166.4+ MB


In [72]:
df.isna().sum()

id                        0
order_id                  0
driver_id                 0
driver_action             0
lat                       0
lng                       0
created_at          1557739
updated_at          1557739
order_origin              0
trip_origin               0
trip_destination          0
trip_Start_time        2022
trip_end_time             0
dtype: int64

### Clean the dataframe

Drop 'created_at' and 'updated_at' -- all the entry is empty.

In [74]:
df.shape

(1557739, 13)

In [79]:
df.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at,order_origin,trip_origin,trip_destination,trip_Start_time,trip_end_time
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


### Duplicate Checks

In [83]:
overview = Overview(df)
prep = PreProcess(df)

INFO:logger:Successfully Instantiated Outlier Class Object


In [85]:
# Duplicate check 
overview.number_of_duplicates(df)

INFO:logger:Number of duplicated fields calculated


(0, 13)


In [82]:
df.describe()

,id,order_id,driver_id,lat,lng,created_at,updated_at
count,1.557739e+06,1.557739e+06,1.557739e+06,1.557739e+06,1.557739e+06,0.0,0.0
mean,7.788709e+05,4.191833e+05,2.341505e+05,6.536031e+00,3.378874e+00,NaN,NaN
std,4.496808e+05,1.444531e+04,2.270476e+04,5.997423e-02,5.317532e-02,NaN,NaN
min,1.000000e+00,3.920010e+05,1.219810e+05,6.409333e+00,3.076561e+00,NaN,NaN
25%,3.894365e+05,4.067650e+05,2.429970e+05,6.498694e+00,3.348810e+00,NaN,NaN
50%,7.788710e+05,4.216870e+05,2.435890e+05,6.544247e+00,3.363504e+00,NaN,NaN
75%,1.168306e+06,4.293130e+05,2.440560e+05,6.593276e+00,3.385176e+00,NaN,NaN
max,1.557740e+06,5.179480e+05,2.478770e+05,6.735270e+00,3.858373e+00,NaN,NaN


In [87]:
clean_df = prep.drop_column(df,['created_at', 'updated_at'])
clean_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1557739 entries, 0 to 1557738
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1557739 non-null  int64  
 1   order_id          1557739 non-null  int64  
 2   driver_id         1557739 non-null  int64  
 3   driver_action     1557739 non-null  object 
 4   lat               1557739 non-null  float64
 5   lng               1557739 non-null  float64
 6   order_origin      1557739 non-null  object 
 7   trip_origin       1557739 non-null  object 
 8   trip_destination  1557739 non-null  object 
 9   trip_Start_time   1555717 non-null  object 
 10  trip_end_time     1557739 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 142.6+ MB


In [88]:
clean_df.head()

,id,order_id,driver_id,driver_action,lat,lng,order_origin,trip_origin,trip_destination,trip_Start_time,trip_end_time
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
